In [100]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

In [101]:

def extract_data(file_path):
    with open(file_path, 'r') as f:
        content = f.readlines()

    activity_log_line = None
    for line in content:
        if "Activity log file:" in line:
            activity_log_line = line
            break

    if activity_log_line:
        # Extract the file name from the line
        file_name = re.findall(r"Activity log file:\s*(.*?\.dat)", activity_log_line)[0]

        # Extract machine_number (S414) from the file name
        machine_number = re.findall(r'S(\d+)', file_name)[0]

        # Extract control_number (CTRL4067) from the file name
        control_number = re.findall(r'CTRL(\d+)', file_name)[0]

    # Find the line that contains "Interrupt"
    interrupt_line = None
    for line in content:
        if "Interrupt" in line:
            interrupt_line = line
            break

    s_vsense = None
    s_ovp = None
    for line in content:
        if "S_VSENSE" in line:
            s_vsense = re.findall(r"(-?\d+\.?\d*)", line)[0]
        if "S_OVP" in line:
            s_ovp = re.findall(r"(-?\d+\.?\d*)", line)[0]

    if interrupt_line:
        # Extract timestamp
        timestamp = re.findall(r'\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}', interrupt_line)[0]

        # Find the last test step before the interruption
        test_step = None
        for i in range(content.index(interrupt_line) - 1, 0, -1):
            if "Test Step:" in content[i]:
                test_step = re.findall(r"\*+\s*Test Step:\s*(.*?)\s*\*+\s*", content[i])[0]
                break

        return {
            'machine_number': machine_number,
            'control_number': control_number,
            'timestamp': timestamp,
            'test_step': test_step,
            'S_VSENSE': s_vsense,
            'S_OVP': s_ovp
        }
    else:
        return None

In [102]:

def process_folder(folder_path):
    all_data = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".dat"):
            file_path = os.path.join(folder_path, filename)
            data = extract_data(file_path)
            if data:
                all_data.append(data)

    return pd.DataFrame(all_data)

In [103]:
if True:
    folder_path = "data-voltages"  # Replace with the actual path to your folder
    for product_name in os.listdir(folder_path):
        print(product_name)
        product_folder_path = os.path.join(folder_path, product_name)
        df = process_folder(product_folder_path)
        df.to_csv(f"output-voltages/{product_name}-output.csv", index=False)  # Save data to CSV

Breithorn
Strix_Halo


In [104]:
import altair as alt

In [105]:
def visualise(product_name):
    df = pd.read_csv(f"output-voltages/{product_name}-output.csv", parse_dates=['timestamp'])
    df_melted = df.melt(
        id_vars=['machine_number', 'control_number', 'timestamp', 'test_step'],
        value_vars=['S_VSENSE', 'S_OVP'],
        var_name='metric',
        value_name='value'
    )

    # Create a selection for filtering.
    selection = alt.selection_single(fields=['machine_number', 'control_number', 'test_step'], bind='legend')

    # Create the Altair chart.
    chart = (
    alt.Chart(df_melted)
    .mark_point()
    .encode(
        x=alt.X('timestamp', axis=alt.Axis(title='Timestamp')),
        y=alt.Y('value', axis=alt.Axis(title='Value')),
        color=alt.Color('metric', legend=alt.Legend(title='Metric')),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
        tooltip=['timestamp', 'value', 'metric', 'machine_number', 'control_number', 'test_step']
    )
    .properties(title='S_VSENSE and S_OVP Over Time')
    .add_selection(selection)
    .interactive()
    )

    chart.display()

In [106]:
product = "Breithorn"
visualise(product)

C:\Users\amoshnin\AppData\Local\Temp\ipykernel_42476\1964755902.py:11: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use selection_point instead.
  selection = alt.selection_single(fields=['machine_number', 'control_number', 'test_step'], bind='legend')
C:\Users\amoshnin\AppData\Local\Temp\ipykernel_42476\1964755902.py:25: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  .add_selection(selection)


alt.Chart(...)